In [1]:
import os
import re
import math
import json

from pprint import pprint
from ollama import Client as OllamaClient

from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.utils.extraction import extract_tag_content

MODEL  = "llama3.2:3b"
CLIENT = OllamaClient()

In [2]:
# Define the System Prompt as a constant
REACT_SYSTEM_PROMPT = """
You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
%s
</tools>

Example session:

<question>What's the current temperature in Madrid?</question>
<thought>I need to get the current weather in Madrid</thought>
<tool_call>{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {"temperature": 25, "unit": "celsius"}}</observation>

You then output:

<response>The current temperature in Madrid is 25 degrees Celsius</response>

Additional constraints:

- If the user asks you something unrelated to any of the tools above, answer freely enclosing your answer with <response></response> tags.
"""

In [3]:
@tool
def sum_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.

    Args:
        a (int): The first integer to be summed.
        b (int): The second integer to be summed.

    Returns:
        int: The sum of `a` and `b`.
    """
    return a + b


@tool
def multiply_two_elements(a: int, b: int) -> int:
    """
    Multiplies two integers.

    Args:
        a (int): The first integer to multiply.
        b (int): The second integer to multiply.

    Returns:
        int: The product of `a` and `b`.
    """
    return a * b

@tool
def compute_log(x: int) -> float | str:
    """
    Computes the logarithm of an integer `x` with an optional base.

    Args:
        x (int): The integer value for which the logarithm is computed. Must be greater than 0.

    Returns:
        float: The logarithm of `x` to the specified `base`.
    """
    if x <= 0:
        return "Logarithm is undefined for values less than or equal to 0."
    
    return math.log(x)


available_tools = {
    "sum_two_elements": sum_two_elements,
    "multiply_two_elements": multiply_two_elements,
    "compute_log": compute_log
}

In [4]:
sum_two_elements.name

'sum_two_elements'

In [5]:
pprint(sum_two_elements.fn_signature)

('{"name": "sum_two_elements", "description": "\\n    Computes the sum of two '
 'integers.\\n\\n    Args:\\n        a (int): The first integer to be '
 'summed.\\n        b (int): The second integer to be summed.\\n\\n    '
 'Returns:\\n        int: The sum of `a` and `b`.\\n    ", "parameters": '
 '{"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}')


In [6]:
tools_signature = ""
for tk, tv in available_tools.items():
    tools_signature += tv.fn_signature + ",\n"

# tools_signature = sum_two_elements.fn_signature + ",\n" + multiply_two_elements.fn_signature + ",\n" + compute_log.fn_signature

In [7]:
pprint(tools_signature)

('{"name": "sum_two_elements", "description": "\\n    Computes the sum of two '
 'integers.\\n\\n    Args:\\n        a (int): The first integer to be '
 'summed.\\n        b (int): The second integer to be summed.\\n\\n    '
 'Returns:\\n        int: The sum of `a` and `b`.\\n    ", "parameters": '
 '{"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},\n'
 '{"name": "multiply_two_elements", "description": "\\n    Multiplies two '
 'integers.\\n\\n    Args:\\n        a (int): The first integer to '
 'multiply.\\n        b (int): The second integer to multiply.\\n\\n    '
 'Returns:\\n        int: The product of `a` and `b`.\\n    ", "parameters": '
 '{"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},\n'
 '{"name": "compute_log", "description": "\\n    Computes the logarithm of an '
 'integer `x` with an optional base.\\n\\n    Args:\\n        x (int): The '
 'integer value for which the logarithm is computed. Must be greater than '
 '0.\\n\\n    Returns:\\n        

In [8]:
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT % tools_signature

In [9]:
print(REACT_SYSTEM_PROMPT)


You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"pr

In [10]:
# ReAct Loop Step 1
USER_QUESTION = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result"
chat_history = [
    {
        "role": "system",
        "content": REACT_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{USER_QUESTION}</question>"
    }
]

output = (
    CLIENT.chat(
        messages=chat_history, 
        model=MODEL
    )
    .message.content
)

print(output)

<tool_call>
{"name": "sum_two_elements", "arguments": {"a": 1234, "b": 5678}, "id": 1}
</tool_call>

<tool_call>
{"name": "multiply_two_elements", "arguments": {"a": 6272, "b": 5}, "id": 2}
</tool_call>

<tool_call>
{"name": "compute_log", "arguments": {"x": 31360}, "id": 3}
</tool_call>


In [11]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

In [12]:
# ReAct Loop Step 2 
tool_call = extract_tag_content(output, tag="tool_call")

In [13]:
tool_call

TagContentResult(content=['{"name": "sum_two_elements", "arguments": {"a": 1234, "b": 5678}, "id": 1}', '{"name": "multiply_two_elements", "arguments": {"a": 6272, "b": 5}, "id": 2}', '{"name": "compute_log", "arguments": {"x": 31360}, "id": 3}'], found=True)

In [14]:
tool_call = json.loads(tool_call.content[0])

tool_call

{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 1}

In [15]:
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [16]:
assert tool_result == 1234 + 5678

In [17]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

In [18]:
# ReAct Loop Step 3
output = (
    CLIENT.chat(
        messages=chat_history, 
        model=MODEL
    )
    .message.content
)

print(output)

<tool_call>
{"name": "multiply_two_elements", "arguments": {"a": 6912, "b": 5}, "id": 4}
</tool_call>

<tool_call>
{"name": "compute_log", "arguments": {"x": 34800}, "id": 5}
</tool_call>


In [19]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

In [20]:
# ReAct Loop Step 4
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

print(tool_result, (1234 + 5678) * 5)

assert tool_result == (1234 + 5678) * 5

34560 34560


In [21]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

In [22]:
chat_history

[{'role': 'system',
  'content': '\nYou are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.\nYou are provided with function signatures within <tools></tools> XML tags.\nYou may call one or more functions to assist with the user query. Don\' make assumptions about what values to plug\ninto functions. Pay special attention to the properties \'types\'. You should use those types as in a Python dict.\n\nFor each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:\n\n<tool_call>\n{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}\n</tool_call>\n\nHere are the available tools / actions:\n\n<tools> \n{"name": "sum_two_elements", "description": "\\n    Computes the sum of two integers.\\n\\n    Args:\\n        a (int): The first integer to be summed.\\n        b (int): The second integer to be summed.\\n\\n    Returns:\\n   

In [23]:
# ReAct Loop Step 5
output = (
    CLIENT.chat(
        messages=chat_history, 
        model=MODEL
    )
    .message.content
)

chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)


print(output)

<response>The logarithm of 34560 is approximately 8.654</response>


In [ ]:
# React Loop Step 6
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

IndexError: list index out of range

In [ ]:
assert tool_result == math.log((1234 + 5678) * 5)

In [ ]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

In [ ]:
# ReAct Loop Step 7
output = (
    CLIENT.chat(
        messages=chat_history, 
        model=MODEL
    )
    .message.content
)

print(output)